In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from load_pvoc_data import load_data, TRAIN_LENGTH

In [3]:
BATCH_SIZE = 32
EPOCHS = 20
VALIDATION_SPLIT = 0.3

In [4]:
def preprocessing(img, lbl):
    crop_img = tf.image.central_crop(img, 1)
    resized = tf.image.resize_images(img, (256, 256))
    norm_img = tf.image.per_image_standardization(resized)
    
    one_hot = tf.one_hot(lbl, 20)
    summed = tf.reduce_sum(one_hot, axis=-2)
    multi_hot = tf.where(
        tf.equal(summed, 0), tf.zeros_like(summed, dtype=tf.float32), tf.ones_like(summed, dtype=tf.float32)
    )
    return norm_img, multi_hot

In [5]:
def train_input_fn():
    train_dataset = tf.data.Dataset.from_generator(
        lambda:load_data("train"),
        (tf.uint8, tf.int32),
        (tf.TensorShape([None, None, 3]), tf.TensorShape([None]))
    ).map(preprocessing).shuffle(10000)
    train_dataset = train_dataset.apply(tf.contrib.data.assert_element_shape((
        [256, 256, 3],
        [20]
    )))
    
    val_length = int(VALIDATION_SPLIT * TRAIN_LENGTH)
    val_dataset = train_dataset.take(val_length).apply(
        tf.contrib.data.batch_and_drop_remainder(BATCH_SIZE))
    train_dataset = train_dataset.skip(val_length).apply(
        tf.contrib.data.batch_and_drop_remainder(BATCH_SIZE)).repeat()

    return train_dataset, val_dataset

In [6]:
def test_input_fn():
    test_dataset = tf.data.Dataset.from_generator(
        lambda:load_data("test"),
        (tf.uint8, tf.int32),
        (tf.TensorShape([None, None, 3]), tf.TensorShape([None]))
    )
    test_dataset = test_dataset.map(preprocessing).apply(tf.contrib.data.assert_element_shape((
        [256, 256, 3],
        [20]
    )))
    return test_dataset.apply(tf.contrib.data.batch_and_drop_remainder(BATCH_SIZE))

In [7]:
def residual_block(inputs, filters):
    x = tf.layers.conv2d(inputs=inputs, filters=filters, kernel_size=1,
                         strides=1, padding='same')
    x = tf.nn.relu(tf.layers.batch_normalization(x))
    
    x = tf.layers.conv2d(inputs=inputs, filters=(filters * 2), kernel_size=3,
                         strides=1, padding='same')
    return tf.nn.relu(tf.layers.batch_normalization(x) + inputs)

In [8]:
def darknet_block(inputs, filters, repetitions):
    x = tf.layers.conv2d(inputs=inputs, filters=filters, kernel_size=3,
                         strides=2, padding='same')
    x = tf.nn.relu(tf.layers.batch_normalization(x))
    for i in range(repetitions):
        x = residual_block(x, filters / 2)
    return x

In [9]:
def darknet_model(features, labels, mode):  
    features = tf.cast(features, dtype=tf.float32)
    
    x = tf.layers.conv2d(inputs=features, filters=32, kernel_size=3,
                         strides=1, padding='same')
    x = tf.nn.relu(tf.layers.batch_normalization(x))
    x = darknet_block(x, 64, 1)
    x = darknet_block(x, 128, 2)
    x = darknet_block(x, 256, 8)
    x = darknet_block(x, 512, 8)
    x = darknet_block(x, 1024, 4)
    
    x = tf.layers.average_pooling2d(x, pool_size=8, strides=8)
    x = tf.layers.flatten(x)
    x = tf.layers.dense(inputs=x, units=20, kernel_initializer=tf.truncated_normal_initializer())
    
    classes = tf.where(tf.sigmoid(x) >= 0.5, tf.ones_like(x, dtype=tf.float32), tf.zeros_like(x, dtype=tf.float32))
    correct_prediction = tf.equal(classes, labels)
    acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    acc = tf.identity(acc, name='accuracy_tensor')
    
    predictions = {'classes': classes, 'accuracy': acc}
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=x)
    loss = tf.reduce_mean(loss)
    
    tf.summary.scalar('accuracy', acc)
    tf.summary.scalar('loss', loss)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.contrib.estimator.TowerOptimizer(tf.train.AdamOptimizer(1e-4))
        #optimizer = tf.train.AdamOptimizer(1e-5)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {'accuracy': tf.metrics.accuracy(labels=labels, predictions=classes)}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss,
                                      eval_metric_ops=eval_metric_ops)


In [10]:
tensors_to_log = {'accuracy': 'accuracy_tensor'}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=10)

In [11]:
model = tf.estimator.Estimator(
    model_fn=tf.contrib.estimator.replicate_model_fn(darknet_model),
    model_dir='/tmp/tmpdark', config=tf.estimator.RunConfig(
        save_checkpoints_steps=150, save_summary_steps=10, log_step_count_steps=10
    )
)

INFO:tensorflow:Replicating the `model_fn` across ['/device:GPU:0', '/device:GPU:1'].  Variables are going to be placed on ['/CPU:0'].  Consolidation device is going to be /CPU:0.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpdark', '_task_id': 0, '_num_worker_replicas': 1, '_evaluation_master': '', '_service': None, '_tf_random_seed': None, '_log_step_count_steps': 10, '_save_checkpoints_steps': 150, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_master': '', '_is_chief': True, '_save_summary_steps': 10, '_train_distribute': None, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2a6586def0>}


In [12]:
validation_hook = tf.contrib.learn.monitors.replace_monitors_with_hooks(
    [tf.contrib.learn.monitors.ValidationMonitor(
        input_fn=lambda:train_input_fn()[1], every_n_steps=100, early_stopping_rounds=10
    )],
    model
)[0]

Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.


In [23]:
max_steps = int(((1 - VALIDATION_SPLIT) * TRAIN_LENGTH / BATCH_SIZE) * EPOCHS)
model.train(input_fn=lambda:train_input_fn()[0], hooks=[logging_hook, validation_hook],
            max_steps=max_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpdark/model.ckpt-652
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 653 into /tmp/tmpdark/model.ckpt.
INFO:tensorflow:accuracy = 0.934375
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-10-12:31:29
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpdark/model.ckpt-653
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-10-12:33:31
INFO:tensorflow:Saving dict for global step 653: accuracy = 0.9200879, global_step = 653, loss = 0.23294005
INFO:tensorflow:Validation (step 653): accuracy = 0.9200879, global_step = 653, loss = 0.232940

In [13]:
print(model.evaluate(input_fn=test_input_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-10-12:41:42
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpdark/model.ckpt-954
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-10-12:42:30
INFO:tensorflow:Saving dict for global step 954: accuracy = 0.9018945, global_step = 954, loss = 0.23933859
{'accuracy': 0.9018945, 'global_step': 954, 'loss': 0.23933859}
